In [4]:
import pandas as pd
import os
import csv
import glob

In [5]:
file_location = os.path.join('Raw Data','average_price_by_state_rawdata','*.csv')
print(file_location)

Raw Data\average_price_by_state_rawdata\*.csv


In [6]:
filenames = glob.glob(file_location)
print(filenames)

['Raw Data\\average_price_by_state_rawdata\\State_bdrmcnt1_rawdata.csv', 'Raw Data\\average_price_by_state_rawdata\\State_bdrmcnt2_rawdata.csv', 'Raw Data\\average_price_by_state_rawdata\\State_bdrmcnt3_rawdata.csv', 'Raw Data\\average_price_by_state_rawdata\\State_bdrmcnt4_rawdata.csv', 'Raw Data\\average_price_by_state_rawdata\\State_bdrmcnt5_rawdata.csv', 'Raw Data\\average_price_by_state_rawdata\\State_condo_rawdata.csv']


In [7]:
def create_df(raw_data,df_name):
    filtered_df = raw_data.loc[:,["State","CountyName"]]
    filtered_df["Housing Type"] = df_name
    
    year_list = [2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020]
    for year in year_list:
        sample_df = raw_data.filter(like=str(year), axis=1)
        filtered_df[year] = sample_df.mean(axis=1)

        pd.options.display.float_format = '{:.2f}'.format
    
    clean_df = filtered_df.dropna(how ='any')
    return clean_df

In [8]:
count = 0
name_list = ["Data_1_bed","Data_2_bed","Data_3_bed","Data_4_bed","Data_5_bed","Data_condo"]

#Dataframe to store individual housing data
data = {}

for file in filenames:
    raw_data =  f'rawdata_{str(count)}_df'
    print(raw_data)
    raw_data = pd.read_csv(file)
    
    #Give a name to dataframe dynamically   
    df_name = f'{name_list[count]}'
    #Calling function to extract required columns
    exec('df = create_df(raw_data,df_name)')
    
    #Write individual housing type output to *.csv file   
    output_file = os.path.join('Clean Data','Housing data by values',df_name+'.csv')
    df.to_csv(output_file)
    print(df_name)          
    
    #Adding to dataframe for later use
    data[df_name]=(df)
    
    #Increment counter to pick next raw data file
    count += 1


rawdata_0_df
Data_1_bed
rawdata_1_df
Data_2_bed
rawdata_2_df
Data_3_bed
rawdata_3_df
Data_4_bed
rawdata_4_df
Data_5_bed
rawdata_5_df
Data_condo


In [9]:
#Saving the dictionary with cleaned data into excel sheet
excel_file = os.path.join('Clean Data','Housing data by values','final_cleaned_data.xls')
writer = pd.ExcelWriter(excel_file, engine='xlsxwriter')

#now loop thru and put each on a specific sheet
for sheet, frame in  data.items(): 
    frame.to_excel(writer, sheet_name = sheet,index = False)

writer.save()

In [10]:
cleaned_merged_file = os.path.join('Clean Data','Housing data by values','Housing_data_clean_merged.csv')
merged_df = pd.read_excel(excel_file, sheet_name=None)
xlsx = pd.ExcelFile(excel_file)
data_sheets = []
for sheet in xlsx.sheet_names:
    data_sheets.append(xlsx.parse(sheet))
    House_merged_clean = pd.concat(data_sheets)

House_merged_clean.to_csv(cleaned_merged_file, encoding='utf-8', index=False,header = True)  